# Deep Learning Assignment

1. Loading the input data
2. Import Libraries
3. Setting a path for input directory
4. Create input data frame
5. Train and Test split
6. Create Train and Test generators
7. Models
   1. Model - Base Model
   2. Model - Network Architecture
   3. Model - Activation Fuction
   4. Model - Gradient Estimation
   5. Model - Cost Function
   6. Model - Network Initialization
   7. Model - Epochs
   8. Model - Best Model (All Integrated)
   9. Model - MobileNetV2
8. Conclusion
9. Citation
10. License

## Understand dataset structure and files

* The dataset being used is [Food 101](https://www.kaggle.com/kmader/food41)
* This dataset has 101000 images in total. It's a food dataset with 101 categories(multiclass)
* Each type of food has 750 training samples and 250 test samples
* Note found on the webpage of the dataset :  
  * On purpose, the training images were not cleaned, and thus still contain some amount of noise. This comes mostly in the form of intense colors and sometimes wrong labels. All images were rescaled to have a maximum side length of 512 pixels.
  * The entire dataset is 6GB in size

## 1. Load the Dataset

**images** folder contains 101 folders with 1000 images each.<br>
Each folder contains images of a specific food class.

**meta** folder contains the text files - train.txt and test.txt  
**train.txt** contains the list of images that belong to training set  
**test.txt** contains the list of images that belong to test set  
**classes.txt** contains the list of all classes of food

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 2. Import Libraries

In [4]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report

## 3. Setting path for the input directory

In [5]:
image_dir = Path('../input/food41/images')

## 4. Creating File DataFrame

We are setting the respective paths and labels.

In [6]:
filepaths = list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

images = pd.concat([filepaths, labels], axis=1)

category_samples = []
for category in images['Label'].unique():
    category_slice = images.query("Label == @category")
    category_samples.append(category_slice.sample(100, random_state=1))
image_df = pd.concat(category_samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [7]:
image_df

In [8]:
image_df['Label'].value_counts()

## 5. Train-Test Split

In [9]:
train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

## 6. Creating Generators




This is where we normalise the data. We mention the size, width and all other parameters such that the images can be flipped, zoom, and more to be recognised in a way that is similar to the rest.

In [10]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
   width_shift_range=0.1,
   height_shift_range=0.1,
   rescale=1./255.,
   shear_range=0.2,
   zoom_range=0.2,
   horizontal_flip=True,
   fill_mode='nearest',
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
)

Takes data & label arrays, generates batches of augmented data. Input data. Numpy array of rank 4 or a tuple. If tuple, the first element should contain the images and the second element another numpy array or a list of numpy arrays that gets passed to the output without any modifications.

In [11]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

Below, we are splitting the dataset into train and test by specifying the target size. We are also specifying other parameters like categorical since it is a classification dataset. The train, test and validation sets are used in further steps for our accuracy.
Batch size is how many images are taken in a batch.

In [12]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(128, 128),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=8,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(128, 128),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=8,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(128, 128),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=8,
    shuffle=False
)

## 7. Create Models

### 1. Model - The base model

Neural network layer gives output where it scales the value between layers. Below RelU is already in use. Also we are using sequential for sequence layers. Conv2D is used for image classification. We can see a list of layers below, with sizes starting from 1024 and going in a decending manner. The more the number of layers, the more information can be extracted by the model from the dataset. input shape also mentions the shapes which are consequences of the model's configuration. Shapes are tuples representing how many elements an array or tensor has in each dimension.

The model we are using first is a ADMN. The default network architecture here is random normal. Weights here are gaussian distributions. The loss is the cross entropy loss which has to decrease for us to get a good accuracy.

In [15]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation

from keras.models import Sequential

#add model layers
model = Sequential()
model.add(Conv2D(1024, kernel_size=3, activation= 'relu', input_shape=(128,128, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(512, kernel_size=3, activation= 'relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(256, kernel_size=3, activation= 'relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(256, kernel_size=3, activation= 'relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(256, kernel_size=3, activation= 'relu'))
model.add(Flatten())
model.add(Dense(101, activation='softmax'))
          
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100
)

In [19]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### 2. Model - Network Arcihtecture

In our second model we are taking a reduced layer number, and sizes respectively.We are making use of sequential, Maxpooling and Dense. Dense is a single vector layer. Max pooling is done to in part to help over-fitting by providing an abstracted form of the representation. As well, it reduces the computational cost by reducing the number of parameters to learn and provides basic translation invariance to the internal representation.

In [14]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.models import Sequential
#add model layers
model = Sequential()
model.add(Conv2D(102, kernel_size=3, activation= 'relu', input_shape=(128,128, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(512, kernel_size=3, activation= 'relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(256, kernel_size=3, activation= 'relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(256, kernel_size=3, activation= 'relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(128, kernel_size=3, activation= 'relu'))
model.add(Flatten())
model.add(Dense(101, activation='softmax'))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100
)

### 3. Model - Activation Function

Here, we have tweaked the values using flatten which converges the dataset such as to read it better. Rest has been kept the same for checking accuracy. We can see it increase.

In [18]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation

from keras.models import Sequential

#add model layers
model = Sequential()
model.add(Conv2D(1024, kernel_size=3, activation= 'elu', input_shape=(128,128, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(512, kernel_size=3, activation= 'elu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(256, kernel_size=3, activation= 'elu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(256, kernel_size=3, activation= 'elu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(256, kernel_size=3, activation= 'elu'))
model.add(Flatten())
model.add(Dense(101, activation='softmax'))
          
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100
)

### 4. Model - Gradient Estimation

Here, we have changed the model to SGD.

In [11]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation

from keras.models import Sequential

#add model layers
model = Sequential()
model.add(Conv2D(1024, kernel_size=3, activation= 'relu', input_shape=(128,128, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(512, kernel_size=3, activation= 'relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(256, kernel_size=3, activation= 'relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(256, kernel_size=3, activation= 'relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(256, kernel_size=3, activation= 'relu'))
model.add(Flatten())
model.add(Dense(101, activation='softmax'))

model.compile(
    optimizer='sgd',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100
)

In [12]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### 5. Model - Cost Function

Below we have made use of loss function kl_divergence keeping the rest same for checking if accuracy is better.

In [15]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation

from keras.models import Sequential

#add model layers
model = Sequential()
model.add(Conv2D(1024, kernel_size=3, activation= 'relu', input_shape=(128,128, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(512, kernel_size=3, activation= 'relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(256, kernel_size=3, activation= 'relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(256, kernel_size=3, activation= 'relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(256, kernel_size=3, activation= 'relu'))
model.add(Flatten())
model.add(Dense(101, activation='softmax'))
          
model.compile(
    optimizer='adam',
    loss='kl_divergence',
    metrics=['accuracy']
)


history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100
)

In [16]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### 5. Model - Network Initialization 

Below, we have used Glorot which is different network initialization when compared with random normal. One common initialization scheme for deep NNs is called Glorot (also known as Xavier) Initialization. The idea is to initialize each weight with a small Gaussian value with mean = 0.0 and variance based on the fan-in and fan-out of the weight.

In [ ]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.models import Sequential
#add model layers
model = Sequential()
model.add(Conv2D(102, kernel_size=3, activation= 'relu', input_shape=(128,128, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(512, kernel_size=3, activation= 'relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(256, kernel_size=3, activation= 'relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(256, kernel_size=3, activation= 'relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(128, kernel_size=3, activation= 'relu'))
model.add(Flatten())
model.add(Dense(101,kernel_initializer='glorot_uniform', activation='softmax'))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100
)

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### 7. Model - EPOCHS

Here, we are changing the number of epochs to 50 and check how well the model performs in these 50 epochs.

In [ ]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.models import Sequential
#add model layers
model = Sequential()
model.add(Conv2D(102, kernel_size=3, activation= 'relu', input_shape=(128,128, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(512, kernel_size=3, activation= 'relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(256, kernel_size=3, activation= 'relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(256, kernel_size=3, activation= 'relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(128, kernel_size=3, activation= 'relu'))
model.add(Flatten())
model.add(Dense(101, activation='softmax'))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=50
)

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### 8. Model - Integrated

In [ ]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.models import Sequential
#add model layers
model = Sequential()
model.add(Conv2D(102, kernel_size=3, activation= 'elu', input_shape=(128,128, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(512, kernel_size=3, activation= 'elu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(256, kernel_size=3, activation= 'elu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(256, kernel_size=3, activation= 'elu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(128, kernel_size=3, activation= 'elu'))
model.add(Flatten())
model.add(Dense(101,kernel_initializer='glorot_uniform', activation='softmax'))

model.compile(
    optimizer='sgd',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100
)

### 9. Model - Pre-trained  model (MobileNetV2)

In [13]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(128, 128, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

In [ ]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(101, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)


print(model.summary())

#### Training the MobileNetV2 model

In [15]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

In [16]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### 9.1 Changed Activation Function

In [ ]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='softmax')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='softmax')(x)

outputs = tf.keras.layers.Dense(101, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)


print(model.summary())

In [18]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

In [19]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### 9.2 Changed Optimizer

In [46]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(101, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)


print(model.summary())

In [21]:
model.compile(
    optimizer='sgd',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

In [22]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### 9.3 Changed Cost Function

In [47]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(101, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)


print(model.summary())

In [37]:
model.compile(
    optimizer='adam',
    loss='kl_divergence',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

In [38]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### 9.4 Adding Network Initializer

In [39]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, kernel_initializer='glorot_uniform', activation='relu')(x)

outputs = tf.keras.layers.Dense(101, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)


print(model.summary())

In [40]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

In [41]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### 9.5 Change in Epochs

In [45]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(101, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)


print(model.summary())

In [43]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=50
)

In [44]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## 8.  Conclusion

In conclusion, we started off with a basic model and changed and tuned parameters and trained with separate models with each change. In some cases, the accuracy increased and in some cases it did not. The MobileNetV2 performed better than the basic model used. When changed the parameters of the basic model and integrated, it gave an accuracy of 96%.

## 9. Citation

* [KLDivergence](https://www.tensorflow.org/api_docs/python/tf/keras/losses/KLDivergence)
* [Deep Lizard Keras Tutorial](https://www.youtube.com/watch?v=stWU37L91Yc&list=PLZbbT5o_s2xrfNyHZsM6ufI0iZENK9xgG&index=20)
* [Display Deep Learning Model Training in Keras](https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/)
* [Kaggle](https://www.kaggle.com/illukthegreat/starter-food-images)
* [Keras Starter](https://github.com/rileykwok/Food-Classification/blob/master/Food-101%20Challenge.ipynb)

## 10. License

Copyright [2021] [Vachana Satish Belgavi, Akshata Nanjappa]

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.